# Plausi der Messwerte der Wetterstationen der Wasserschutzpolizei Zürich
Datum: 12.01.2022

**Ausgangslage:**

Hr. Namnick liefert jährlich per Mail die Jahresdaten der Wetterstationen. Bisher haben wir die neu gelieferten Jahresdaten in SAS eingelesen, die Variabelnamen standardisiert und richtig sortiert und letztlich die neuen mit den bisherigen Jahren zeitlich sortiert zusammengehängt.

Odi hat zusätzlich noch ein [**Python-Skript serverseitig**](https://github.com/opendatazurich/ogd-data-processing/blob/main/sid_wapo_wetterstationen/convert_csv.py) generiert um aus der mitgelieferten `utc` das `cet` Datum aus dem Datumsfeld zu berechnen. Mit diesem Notebook können wir das aber gleich beim Update erledigen.

Die Datenaufbereitung habe ich 2022 mit Jupyter Lab gemacht, siehe [**Github**](https://github.com/DonGoginho/myPy/blob/main/update_ogd/update_sid_wapo_wetterstationen.ipynb )



**Dataset auf PROD Datenkatalog**:  https://data.stadt-zuerich.ch/dataset/sid_wapo_wetterstationen

## Einstellungen
### Importiere die notwendigen Packages

In [1]:
#%pip install openpyxl geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly

In [2]:
import pandas as pd
import datetime
import time
import numpy as np

import pivottablejs
from pivottablejs import pivot_ui
import altair as alt
import matplotlib.pyplot as plt
#from datetime import datetime
import geopandas as gpd
import folium 

Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [3]:
pd.options.display.float_format = '{:.1f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import `datetime` und `from datetime import datetime`, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [4]:
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")
print(now, date_today, year_today, month_today,day_today )

2022-01-26 2022-01-26 2022 01 26


Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [5]:
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
      'datenstand: ', 
      selectedMonat,
     int_times)


2022 1 datenstand:  -1 time.struct_time(tm_year=2022, tm_mon=1, tm_mday=26, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=26, tm_isdst=-1)


## Importiere die bereits veröffentlichten Zeitreihen der Messstationen 

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**:  https://data.integ.stadt-zuerich.ch/dataset/sid_wapo_wetterstationen

**Dataset auf PROD Datenkatalog**:  https://data.stadt-zuerich.ch/dataset/sid_wapo_wetterstationen

In [6]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "lokal";
print(status)

lokal


In [7]:
# Filepath
if status == "www":
    #fp = r"//szh.loc/ssz/applikationen/OGD_Dropzone/DWH/bev_monat_bestand_quartier_geschl_ag_herkunft_od3250/BEV325OD3250.csv"
    fp_my = r"https://data.stadt-zuerich.ch/dataset/sid_wapo_wetterstationen/download/messwerte_mythenquai_2007-2021.csv"
    fp_tb = r"https://data.stadt-zuerich.ch/dataset/sid_wapo_wetterstationen/download/messwerte_tiefenbrunnen_2007-2021.csv"
else:
    fp_my = r"\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_2007-2021.csv"
    fp_tb = r"\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_tiefenbrunnen_2007-2021.csv"


print(fp_my, fp_tb)

\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_2007-2021.csv \\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_tiefenbrunnen_2007-2021.csv


Ich importiere die Datumsfelder bewussts als STRINGS. Nicht im Datumsformat.
Dies erlaubt später ein einfaches zusammmensetzen der neuen zu den bestehenden Daten.

- [**Cheat Sheet**](https://strftime.org/)
- Python documentation for **strptime**: [string **parse** time, Python 3](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)
- Python documentation for strptime/**strftime**: [string **format** time,Python 3](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

In [8]:
# Read the data
df_zeitreihe_my = pd.read_csv(
    fp_my
    , sep=','
    , low_memory=False    
    )

print('done')

done


In [9]:
# Read the data
df_zeitreihe_tb = pd.read_csv(
    fp_tb 
    ,sep=','
    ,low_memory=False    
    )

print('done')

done


Wandle timestamp_utc noch in ein Datetime-Format um. Könnte auch beim einlesen gemacht werden.

In [10]:
df_zeitreihe_my['timestamp_utc'] = pd.to_datetime(df_zeitreihe_my['timestamp_utc'], utc=True)
df_zeitreihe_tb['timestamp_utc'] = pd.to_datetime(df_zeitreihe_tb['timestamp_utc'], utc=True)

In der Folge ein paar erste Tests:
 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [11]:
#data2bextended_my.head(2)
df_zeitreihe_my.dtypes

timestamp_utc              datetime64[ns, UTC]
timestamp_cet                           object
air_temperature                        float64
water_temperature                      float64
wind_gust_max_10min                    float64
wind_speed_avg_10min                   float64
wind_force_avg_10min                   float64
wind_direction                           int64
windchill                              float64
barometric_pressure_qfe                float64
precipitation                          float64
dew_point                              float64
global_radiation                       float64
humidity                                 int64
water_level                            float64
dtype: object

In [12]:
#data2bextended_tb.head(2)
df_zeitreihe_tb.dtypes

timestamp_utc              datetime64[ns, UTC]
timestamp_cet                           object
air_temperature                        float64
water_temperature                      float64
wind_gust_max_10min                    float64
wind_speed_avg_10min                   float64
wind_force_avg_10min                   float64
wind_direction                           int64
windchill                              float64
barometric_pressure_qfe                float64
precipitation                          float64
dew_point                              float64
global_radiation                       float64
humidity                                 int64
water_level                            float64
dtype: object

In [13]:
df_zeitreihe_my.shape

(759119, 15)

In [14]:
df_zeitreihe_tb.shape

(757537, 15)

Beschreibe einzelne Attribute

In [15]:
df_zeitreihe_my.describe()
#data2bextended_tb.describe()

,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
count,759119.0,658722.0,759119.0,759119.0,759119.0,759119.0,759119.0,754378.0,658722.0,759119.0,658722.0,759119.0,658722.0
mean,11.5,13.4,3.5,1.9,1.8,184.5,10.4,975.6,0.0,6.8,137.7,75.2,405.9
std,8.0,6.8,2.6,1.4,1.3,106.0,8.6,17.8,0.2,6.5,299.0,16.4,0.1
min,-13.4,2.4,-0.1,0.0,0.0,0.0,-25.6,930.7,0.0,-17.2,0.0,16.0,405.2
25%,5.2,6.4,1.7,0.9,1.0,103.0,3.7,966.1,0.0,1.9,0.0,65.0,405.9
50%,11.3,13.1,2.9,1.6,1.7,176.0,10.3,970.9,0.0,6.8,7.0,79.0,405.9
75%,17.5,19.6,4.7,2.6,2.4,286.0,16.8,977.2,0.0,12.1,161.0,87.0,406.0
max,37.7,28.0,32.0,17.1,16.8,360.0,37.8,1037.5,17.0,24.6,4293.0,100.0,406.5


Wie viele Nullwerte gibt es im Datensatz?

In [16]:
df_zeitreihe_my.isnull().sum()

timestamp_utc                   0
timestamp_cet                   0
air_temperature                 0
water_temperature          100397
wind_gust_max_10min             0
wind_speed_avg_10min            0
wind_force_avg_10min            0
wind_direction                  0
windchill                       0
barometric_pressure_qfe      4741
precipitation              100397
dew_point                       0
global_radiation           100397
humidity                        0
water_level                100397
dtype: int64

In [17]:
df_zeitreihe_tb.isnull().sum()

timestamp_utc                   0
timestamp_cet                   0
air_temperature                 0
water_temperature               0
wind_gust_max_10min             0
wind_speed_avg_10min            0
wind_force_avg_10min            0
wind_direction                  0
windchill                       0
barometric_pressure_qfe     52549
precipitation              757537
dew_point                       0
global_radiation           757537
humidity                        0
water_level                757537
dtype: int64

## Grafische Auswertungen
### Verwende das Datum als Index

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. 
**To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame**

In [18]:
df_zeitreihe_my = df_zeitreihe_my.set_index("timestamp_utc")

In [19]:
df_zeitreihe_my.info()
df_zeitreihe_my.index.year.unique()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 759119 entries, 2007-04-22 19:20:00+00:00 to 2021-12-31 23:00:00+00:00
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   timestamp_cet            759119 non-null  object 
 1   air_temperature          759119 non-null  float64
 2   water_temperature        658722 non-null  float64
 3   wind_gust_max_10min      759119 non-null  float64
 4   wind_speed_avg_10min     759119 non-null  float64
 5   wind_force_avg_10min     759119 non-null  float64
 6   wind_direction           759119 non-null  int64  
 7   windchill                759119 non-null  float64
 8   barometric_pressure_qfe  754378 non-null  float64
 9   precipitation            658722 non-null  float64
 10  dew_point                759119 non-null  float64
 11  global_radiation         658722 non-null  float64
 12  humidity                 759119 non-null  int64  
 13  water_level  

Int64Index([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
            2021],
           dtype='int64', name='timestamp_utc')

In [20]:
df_zeitreihe_tb = df_zeitreihe_tb.set_index("timestamp_utc")


In [21]:
df_zeitreihe_tb.info()
df_zeitreihe_tb.index.year.unique()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 757537 entries, 2007-04-15 09:30:00+00:00 to 2021-12-31 23:00:00+00:00
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   timestamp_cet            757537 non-null  object 
 1   air_temperature          757537 non-null  float64
 2   water_temperature        757537 non-null  float64
 3   wind_gust_max_10min      757537 non-null  float64
 4   wind_speed_avg_10min     757537 non-null  float64
 5   wind_force_avg_10min     757537 non-null  float64
 6   wind_direction           757537 non-null  int64  
 7   windchill                757537 non-null  float64
 8   barometric_pressure_qfe  704988 non-null  float64
 9   precipitation            0 non-null       float64
 10  dew_point                757537 non-null  float64
 11  global_radiation         0 non-null       float64
 12  humidity                 757537 non-null  int64  
 13  water_level  

Int64Index([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
            2021],
           dtype='int64', name='timestamp_utc')

In [22]:
sommerzeitwechsel_tb = df_zeitreihe_tb.loc["2021-03-28 01":"2021-03-28 05"].resample("H").mean()
sommerzeitwechsel_tb

,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
timestamp_utc,,,,,,,,,,,,,
2021-03-28 01:00:00+00:00,4.3,6.5,0.5,0.1,0.2,57.0,4.3,NaN,NaN,-0.0,NaN,73.5,NaN
2021-03-28 02:00:00+00:00,3.6,6.6,0.2,0.1,0.2,50.5,3.6,NaN,NaN,-0.3,NaN,76.0,NaN
2021-03-28 03:00:00+00:00,3.0,6.7,1.1,0.4,0.3,64.2,3.0,NaN,NaN,-0.4,NaN,78.2,NaN
2021-03-28 04:00:00+00:00,2.6,6.7,0.7,0.1,0.2,35.5,2.6,NaN,NaN,-0.2,NaN,81.3,NaN
2021-03-28 05:00:00+00:00,2.4,6.6,0.1,0.0,0.0,73.7,2.4,NaN,NaN,0.1,NaN,84.3,NaN


In [23]:
sommerzeitwechsel_my = df_zeitreihe_my.loc["2021-03-28 01":"2021-03-28 05"].resample("H").mean()
sommerzeitwechsel_my

,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
timestamp_utc,,,,,,,,,,,,,
2021-03-28 01:00:00+00:00,5.0,NaN,2.0,1.7,1.9,120.1,4.8,1030.9,NaN,1.4,NaN,77.7,NaN
2021-03-28 02:00:00+00:00,4.3,NaN,2.0,1.7,1.8,150.0,4.2,1030.5,NaN,1.2,NaN,80.7,NaN
2021-03-28 03:00:00+00:00,3.9,NaN,1.3,0.9,1.0,144.7,3.9,1030.3,NaN,1.1,NaN,81.5,NaN
2021-03-28 04:00:00+00:00,3.2,NaN,1.3,0.8,0.7,211.7,3.2,1030.5,NaN,0.9,NaN,84.3,NaN
2021-03-28 05:00:00+00:00,3.4,NaN,2.2,1.5,1.3,288.3,3.1,1031.2,NaN,0.6,NaN,82.3,NaN


In [24]:
# first we create the sums per week
weekly_means = df_zeitreihe_tb.resample("W").mean()
# then we generate the weekly means for each quarter
quarterly_means = df_zeitreihe_tb.resample("Q").mean()
quarterly_median = df_zeitreihe_tb.resample("Q").median()
# for readability we'll revert the values back to integers
#weekly_means.dropna().astype(int).head(5)
#weekly_means
#quarterly_means
#quarterly_median

### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

Daten zu gross hierfür... Stürzt ab

In [25]:
#from pivottablejs import pivot_ui

#pivot_ui(df_zeitreihe_tb)

### Zeitpunkte und Zeiträume abfragen

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

### Aggriere Werte nach Zeitausschnitten

Mit den Funktionen zur Zeit kann einfach zwischen Stunden, Tagen, Monaten, etc. gewechselt und aggregiert werden.

Hier z.B. ob es nach dem Zusammenhängen der Jahresbestände flüssige Übergänge gibt oder ob etwas verdächtig aussieht.


In [26]:
#data2betested_my.loc["2017-06-30"]
df_zeitreihe_tb.loc["2020-12-31 21:50":"2021-01-01 04:30"].resample("H").mean()
df_zeitreihe_my.loc["2020-12-31 21:50":"2021-01-01 04:30"].resample("H").mean()

,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
timestamp_utc,,,,,,,,,,,,,
2020-12-31 21:00:00+00:00,1.6,NaN,3.0,1.8,2.0,90.0,0.8,1005.0,NaN,0.1,NaN,90.0,NaN
2020-12-31 22:00:00+00:00,2.1,NaN,2.4,1.6,1.8,275.0,1.8,1005.0,NaN,0.5,NaN,89.3,NaN
2020-12-31 23:00:00+00:00,2.7,NaN,2.1,1.0,1.0,194.7,2.3,1004.6,NaN,1.3,NaN,90.6,NaN
2021-01-01 00:00:00+00:00,2.3,NaN,2.3,1.5,1.5,299.3,1.9,1004.6,NaN,1.1,NaN,91.5,NaN
2021-01-01 01:00:00+00:00,2.5,NaN,2.8,1.4,1.5,339.0,1.8,1004.9,NaN,1.4,NaN,92.0,NaN
2021-01-01 02:00:00+00:00,2.5,NaN,2.4,1.3,1.3,332.5,1.7,1005.2,NaN,1.4,NaN,92.7,NaN
2021-01-01 03:00:00+00:00,2.7,NaN,1.2,0.6,0.7,150.3,2.7,1005.2,NaN,1.6,NaN,91.8,NaN
2021-01-01 04:00:00+00:00,2.8,NaN,2.1,1.4,1.2,222.5,2.7,1005.4,NaN,1.5,NaN,90.8,NaN


In [27]:
# first we create the sums per week
weekly_medians = df_zeitreihe_tb.resample("W").median()
# then we generate the weekly means for each quarter
quarterly_medians = df_zeitreihe_tb.resample("Q").median()
# for readability we'll revert the values back to integers
quarterly_medians.dropna().astype(int).head(2)

,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
timestamp_utc,,,,,,,,,,,,,


### Visualisierungen nach Zeitausschnitten

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [28]:
#weekly_medians.dtypes
days_tb = df_zeitreihe_tb.resample("D").median()
months_tb = df_zeitreihe_tb.resample("M").median()
years_tb = df_zeitreihe_tb.resample("Y").median()
years_tb.dtypes

air_temperature            float64
water_temperature          float64
wind_gust_max_10min        float64
wind_speed_avg_10min       float64
wind_force_avg_10min       float64
wind_direction             float64
windchill                  float64
barometric_pressure_qfe    float64
precipitation              float64
dew_point                  float64
global_radiation           float64
humidity                   float64
water_level                float64
dtype: object

**----------------Testing----------- **

**Ich möchte eine Grafik, in der die Jahre farblich eingefärbt werden und der Verlauf jedes Jahres dargestellt wird**

In [29]:
# add a column for month names and month number
months_tb['month_number'] = months_tb.index.month
months_tb['month_name'] = months_tb.index.month_name()
months_tb['year'] =months_tb.index.year

#months_tb.head(2)

In [30]:
months_tb[['air_temperature','water_temperature','month_number','month_name','year']].reset_index().head(2)

,timestamp_utc,air_temperature,water_temperature,month_number,month_name,year
0,2007-04-30 00:00:00+00:00,17.0,15.2,4,April,2007
1,2007-05-31 00:00:00+00:00,15.7,16.1,5,May,2007


In [31]:
chart1 = alt.Chart(months_tb[['air_temperature','water_temperature','month_number','month_name','year']].loc["2010-12-31 21:50":"2021-01-01 04:30"].reset_index()).mark_line( strokeWidth=1.5, opacity=0.9).encode(
    x='month_number',
    y='air_temperature',
    color=alt.Color('year', legend=alt.Legend(title="Jahre"), scale=alt.Scale(scheme='cividis'))
).properties(width=800, height=400).interactive()

chart2 = alt.Chart(months_tb[['air_temperature','water_temperature','month_number','month_name','year']].loc["2010-12-31 21:50":"2021-01-01 04:30"].reset_index()).mark_line(interpolate="basis", opacity=0.6, strokeWidth=0.8).encode(
    x='month_number',
    y='air_temperature',
    color=alt.Color('year', legend=alt.Legend(title="Jahre"), scale=alt.Scale(scheme='viridis'))
).properties(width=800, height=400).interactive()

chart1 + chart2

alt.LayerChart(...)

Weitere custimisation, siehe https://altair-viz.github.io/user_guide/customization.html


**--------------- end test ----------------------**

In [32]:
months_tb[['air_temperature','water_temperature']].head(2)

,air_temperature,water_temperature
timestamp_utc,,
2007-04-30 00:00:00+00:00,17.0,15.2
2007-05-31 00:00:00+00:00,15.7,16.1


In [33]:
chart1 = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1, opacity=0.25).encode(
    x='timestamp_utc',
    y='value',
    color='variable',
).properties(width=800, height=400)

chart2 = alt.Chart(years_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(interpolate="basis", opacity=1).encode(
    x='timestamp_utc',
    y='value',
    color='variable',
)

chart1 + chart2

alt.LayerChart(...)

In [34]:
rolling = months_tb[['air_temperature','water_temperature']].rolling(60, center=True, win_type="triang").mean()

chart1 = alt.Chart(rolling.reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1.5, opacity=1).encode(
    x='timestamp_utc', y='value', color='variable',
).properties(width=800, height=400)

# same as the two charts in previous code cell, except more transparent
chart2 = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1.6, opacity=0.25).encode(
    x='timestamp_utc', y='value', color='variable',
)

chart3 = alt.Chart(years_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(interpolate="basis", opacity=0.25).encode(
    x='timestamp_utc', y='value', color='variable',
)

chart1 + chart2 + chart3

alt.LayerChart(...)

In [36]:
# add a column for month names and month number
months_tb['month_number'] = months_tb.index.month
months_tb['month_name'] = months_tb.index.month_name()
months.head(2)

NameError: name 'months' is not defined

In [37]:
grp_months_tb = months_tb[['air_temperature','water_temperature', 'month_number', 'month_name']].groupby("month_name").mean()

In [38]:
alt.Chart(grp_months_tb.reset_index()).mark_bar(width=20).encode(
    x='month_number:O',
    y='air_temperature:Q',
    color='month_name:O'
).properties(width=300, height=300).interactive()

alt.Chart(...)

In [39]:
months_tb = df_zeitreihe_tb.resample("M").median()
years_tb = df_zeitreihe_tb.resample("Y").median()
#months = df_zeitreihe_tb.resample("M").min()
#years = df_zeitreihe_tb.resample("Y").min()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(years_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_area(interpolate="basis").encode(
    x = alt.X('timestamp_utc:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1).encode(
    x = alt.X('timestamp_utc:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

alt.VConcatChart(...)

In [40]:
days_tb = df_zeitreihe_tb.loc["2017-01-01 00:00":"2022-01-01 00:00"].dropna(axis=1).resample("D").median()
months_tb =df_zeitreihe_tb[['air_temperature','water_temperature']].dropna(axis=1).loc["2017-01-01 00:00":"2022-01-01 00:00"].resample("M").median()

years_tb = df_zeitreihe_tb.resample("Y").median()
#months = df_zeitreihe_tb.resample("M").min()
#years = df_zeitreihe_tb.resample("Y").min()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(months_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_area(interpolate="basis").encode(
    x = alt.X('timestamp_utc:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(days_tb[['air_temperature','water_temperature']].reset_index().melt("timestamp_utc")).mark_line(strokeWidth=1).encode(
    x = alt.X('timestamp_utc:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

alt.VConcatChart(...)

In [41]:
#df_zeitreihe_tb[['air_temperature','water_temperature']].dropna(axis=1).loc["2018-12-31 21:50":"2022-01-01 04:30"].resample("M").median()

## Datenexport

Wenn alle Tests positiv und plausibel sind, kann die neu updateten Datasets als csv exportiert und später veröffentlicht werden.

Checke zuerst kurz, ob im Verlauf der Plausis etwas falsches reingerutscht ist

In [42]:
df_zeitreihe_tb.shape
#df_zeitreihe_tb.describe()

(757537, 14)

In [43]:
df_zeitreihe_my.shape
#df_zeitreihe_my.describe()

(759119, 14)

### Exportpfade definieren:

#### Dynamisch berechnete Min- und Max-Jahre

In [44]:
#years.index.year
years = df_zeitreihe_tb.resample("Y").median()
min_year = years.index.year.min()
max_year= years.index.year.max()

print(min_year, max_year, r"\\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_"+str(min_year)+"-"+str(max_year)+".csv")

2007 2021 \\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_2007-2021.csv


#### Pfade zusammensetzen

In [45]:
export_fp_my = r"\\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_mythenquai_"+str(min_year)+"-"+str(max_year)+".csv" #Mythenquai
export_fp_tb= r"\\szh\ssz\applikationen\OGD\Daten\Quelldaten\SID\WAPO\02_veroeffentlichte_zeitreihe\messwerte_tiefenbrunnen_"+str(min_year)+"-"+str(max_year)+".csv" #Tiefenbrunnen

#### Exportoptionen festlegen

Optionen:
`DataFrame.to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', errors='strict', storage_options=None)`

### Export ausführen

1. Indexe aufheben. Dabei verliert der indexierte Wert das Iso-Datumsformat
2. Zeitformat bei zuvor indexiertem Datum nochmals ISO-konform definieren: Dies ist nur für utc wichtig, da cet nie verwendet wurde zuvor und weiterhin als String in ISO-Format vorliegt.

##### Mythenquai


In [46]:
df_my = df_zeitreihe_my.reset_index()
df_my['timestamp_utc'] = df_my.timestamp_utc.apply(datetime.datetime.isoformat)
df_my.head(2)

,timestamp_utc,timestamp_cet,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
0,2007-04-22T19:20:00+00:00,2007-04-22T21:20:00+02:00,18.9,15.2,1.6,0.7,0.7,321,18.9,973.5,0.0,4.4,3.0,38,405.9
1,2007-04-22T19:30:00+00:00,2007-04-22T21:30:00+02:00,18.1,15.2,1.3,0.8,0.8,346,18.1,973.7,0.0,4.8,3.0,41,405.9


In [49]:
df_my.to_csv(export_fp_my
             , sep=','
             , encoding='utf-8-sig'
             ,index=False
                      )

##### Tiefenbrunnen

In [51]:
df_tb = df_zeitreihe_tb.reset_index()

# Achtung: Reihenfolge spielt hier eine Rolle. 
df_tb['timestamp_utc'] = df_tb.timestamp_utc.apply(datetime.datetime.isoformat)
df_tb.head(2)

,timestamp_utc,timestamp_cet,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level
0,2007-04-15T09:30:00+00:00,2007-04-15T11:30:00+02:00,17.4,11.9,0.6,0.0,0.0,154,17.4,973.2,NaN,9.3,NaN,59,NaN
1,2007-04-15T09:40:00+00:00,2007-04-15T11:40:00+02:00,16.7,12.1,0.5,0.1,0.1,150,16.7,973.3,NaN,10.9,NaN,69,NaN


In [52]:
df_tb.to_csv(export_fp_tb
             , sep=','
             , encoding='utf-8-sig'
             ,index=False
            )

### Zeitformatierungen

Alternativ könnte man die `timestamp_cet`beim den Importen noch nicht machen, resp. bei den bestehenden Daten droppen und erst ganz am Schluss alles berechnen.

Aktuell gehe ich so vor:
1. Die neuen Daten importieren --> aus der Datumsangabe --> als utc parsen. Danach cet berechnen
2. Die bisherigen Daten importieren --> die Datumsangaben aber nur als String importieren. Mit utc wird später gerechnet. cet nicht. Am Schluss speichere ich es eigentlich als String, aber merkt man nicht, weil der Export ohne "" bei Strings kommt.

